# Team 42 Final Project: Tuberculosis Detection

*TODO:
Identify and motivate the problem that you will address. Identify and describe the data (and data
sources) that will support deep learning to address the problem. Your problem may not be necessarily directly related to business, but you need to discuss its relevance in practice and potential business applications and impact.*


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import models,transforms,datasets
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder

## Load the Data and Visualization

*TODO: Specify how you will integrate these data to produce the format required for deep learning, like
what we have discussed in cases and problems.*

In [ ]:
# Load data from google drive
from google.colab import drive
drive.mount('/content/drive')

import zipfile

zip_file_path = '/content/drive/MyDrive/chest_xray_dataset.zip'
extract_folder = '/content/xray_folder/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data transform to convert data to a tensor, resize the image to 256x256, and apply normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Prepare data
dataset_path = "/content/xray_folder/TB_Chest_Radiography_Database"

dataset = ImageFolder(dataset_path, transform=transform)

# Seperate to 80% training and 20% testing
train_ratio = 0.8
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 64
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# obtain one batch of training images
dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.numpy() # convert images to numpy for display
images.shape # (number of examples: 128, number of channels: 3, pixel sizes: 256x256)

(64, 3, 256, 256)

## Define the Network
*TODO: Specify the type of deep learning models built. Discuss your choices for deep learning model architecture and hyperparameters: what are
alternatives, and what are the pros and cons.*

### CNN Model 1 & 2

1 layer with max pooling; ReLu activation; 1 hidden layer; no Drop out layer; Sigmoid activation


In [ ]:
class Model1Trainer:
    def __init__(self, learning_rate=0.01, num_epochs=2, batch_size=64):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._init_model().to(self.device)
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def _init_model(self):
        class CNN(nn.Module):
            def __init__(self):
                super(CNN, self).__init__()
                self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
                self.relu1 = nn.ReLU()
                self.pool1 = nn.MaxPool2d(2, 2)
                self.fc1 = nn.Linear(16 * 128 * 128, 128)  # Adjust input size based on your image size
                self.relu2 = nn.ReLU()
                self.fc2 = nn.Linear(128, 1)

            def forward(self, x):
                x = self.conv1(x)
                x = self.relu1(x)
                x = self.pool1(x)
                x = x.view(x.size(0), -1)
                x = self.fc1(x)
                x = self.relu2(x)
                x = self.fc2(x)
                return x
        return CNN()

    def train(self, train_dataset):
        trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.num_epochs):
            self.model.train()
            running_loss = 0.0

            for batch_idx, (images, labels) in enumerate(trainloader):
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels.unsqueeze(1).float())
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

                if batch_idx % 10 == 0:
                    print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Batch [{batch_idx}/{len(trainloader)}] - Loss: {loss.item()}")

            print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Loss: {running_loss / len(trainloader)}")



In [ ]:
class Model2Trainer:
    def __init__(self, learning_rate=0.01, num_epochs=2, batch_size=64):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._init_model().to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.BCELoss()

    def _init_model(self):
        class CNN(nn.Module):
            def __init__(self):
                super(CNN, self).__init__()

                self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
                self.relu1 = nn.ReLU()
                self.pool1 = nn.MaxPool2d(2, 2)
                self.fc1 = nn.Linear(16 * 128 * 128, 128)
                self.relu2 = nn.ReLU()
                self.fc2 = nn.Linear(128, 1)
                self.sigmoid = nn.Sigmoid()

            def forward(self, x):
                x = self.conv1(x)
                x = self.relu1(x)
                x = self.pool1(x)
                x = x.view(x.size(0), -1)
                x = self.fc1(x)
                x = self.relu2(x)
                x = self.fc2(x)
                x = self.sigmoid(x)  # Applying sigmoid activation
                return x
        return CNN()

    def train(self, train_dataset):
        trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.num_epochs):
            self.model.train()
            running_loss = 0.0

            for batch_idx, (images, labels) in enumerate(trainloader):
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels.unsqueeze(1).float())
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

                if batch_idx % 10 == 0:
                    print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Batch [{batch_idx}/{len(trainloader)}] - Loss: {loss.item()}")

            print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Loss: {running_loss / len(trainloader)}")

### CNN Model 3

3 layer with max pooling; ReLu activation; 1 hidden layer; Drop out layer; Sigmoid activation

In [ ]:
class Model3Trainer:
    def __init__(self, learning_rate=0.01, num_epochs=2, batch_size=64):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._init_model().to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.BCEWithLogitsLoss()

    def _init_model(self):
        class CNN(nn.Module):
            def __init__(self):
                super(CNN, self).__init__()

                self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
                self.relu1 = nn.ReLU()
                self.pool1 = nn.MaxPool2d(2, 2)

                self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
                self.relu2 = nn.ReLU()
                self.pool2 = nn.MaxPool2d(2, 2)

                self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
                self.relu3 = nn.ReLU()
                self.pool3 = nn.MaxPool2d(2, 2)

                self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Adjust input size based on your image size
                self.relu4 = nn.ReLU()
                self.fc2 = nn.Linear(128, 1)

            def forward(self, x):
                x = self.conv1(x)
                x = self.relu1(x)
                x = self.pool1(x)

                x = self.conv2(x)
                x = self.relu2(x)
                x = self.pool2(x)

                x = self.conv3(x)
                x = self.relu3(x)
                x = self.pool3(x)

                x = x.view(x.size(0), -1)
                x = self.fc1(x)
                x = self.relu4(x)
                x = self.fc2(x)
                return x

        return CNN()

    def train(self, train_dataset):
        trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.num_epochs):
            self.model.train()
            running_loss = 0.0

            for batch_idx, (images, labels) in enumerate(trainloader):
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels.unsqueeze(1).float())
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

                if batch_idx % 10 == 0:
                    print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Batch [{batch_idx}/{len(trainloader)}] - Loss: {loss.item()}")

            print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Loss: {running_loss / len(trainloader)}")


## Model 4: model 1 add a drop out

In [ ]:
class Model4Trainer:
    def __init__(self, learning_rate=0.01, num_epochs=2, batch_size=64):
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._init_model().to(self.device)
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def _init_model(self):
        class CNN(nn.Module):
            def __init__(self):
                super(CNN, self).__init__()
                self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
                self.relu1 = nn.ReLU()
                self.pool1 = nn.MaxPool2d(2, 2)
                self.fc1 = nn.Linear(16 * 128 * 128, 128)
                self.dropout = nn.Dropout(p=0.5)  # Added dropout layer
                self.relu2 = nn.ReLU()
                self.fc2 = nn.Linear(128, 1)

            def forward(self, x):
                x = self.conv1(x)
                x = self.relu1(x)
                x = self.pool1(x)
                x = x.view(x.size(0), -1)
                x = self.fc1(x)
                x = self.dropout(x)  # Apply dropout
                x = self.relu2(x)
                x = self.fc2(x)
                return x
        return CNN()

    def train(self, train_dataset):
        trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.num_epochs):
            self.model.train()
            running_loss = 0.0

            for batch_idx, (images, labels) in enumerate(trainloader):
                images, labels = images.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels.unsqueeze(1).float())
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

                if batch_idx % 10 == 0:
                    print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Batch [{batch_idx}/{len(trainloader)}] - Loss: {loss.item()}")

            print(f"Epoch [{epoch + 1}/{self.num_epochs}] - Loss: {running_loss / len(trainloader)}")

## Train the Network

In [ ]:
trainer = Model1Trainer(learning_rate=0.01, num_epochs=2, batch_size=64)
trainer.train(train_dataset)

Epoch [1/2] - Batch [0/53] - Loss: 0.6643001437187195
Epoch [1/2] - Batch [10/53] - Loss: 7.272787094116211
Epoch [1/2] - Batch [20/53] - Loss: 4.964140892028809
Epoch [1/2] - Batch [30/53] - Loss: 0.9412527680397034
Epoch [1/2] - Batch [40/53] - Loss: 2.646838426589966
Epoch [1/2] - Batch [50/53] - Loss: 2.0843727588653564
Epoch [1/2] - Loss: 9.888797905718985
Epoch [2/2] - Batch [0/53] - Loss: 3.6248443126678467
Epoch [2/2] - Batch [10/53] - Loss: 0.7811384201049805
Epoch [2/2] - Batch [20/53] - Loss: 0.540007472038269
Epoch [2/2] - Batch [30/53] - Loss: 0.5877604484558105
Epoch [2/2] - Batch [40/53] - Loss: 0.24517227709293365
Epoch [2/2] - Batch [50/53] - Loss: 0.006395240314304829
Epoch [2/2] - Loss: 0.49160264863642944


In [ ]:
trainer2 = Model2Trainer(learning_rate=0.01, num_epochs=2, batch_size=64)
trainer2.train(train_dataset)

Epoch [1/2] - Batch [0/53] - Loss: 0.6829237341880798
Epoch [1/2] - Batch [10/53] - Loss: 18.75
Epoch [1/2] - Batch [20/53] - Loss: 14.0625
Epoch [1/2] - Batch [30/53] - Loss: 18.75
Epoch [1/2] - Batch [40/53] - Loss: 14.0625
Epoch [1/2] - Batch [50/53] - Loss: 12.5
Epoch [1/2] - Loss: 16.55180044781487
Epoch [2/2] - Batch [0/53] - Loss: 15.625
Epoch [2/2] - Batch [10/53] - Loss: 14.0625
Epoch [2/2] - Batch [20/53] - Loss: 14.0625
Epoch [2/2] - Batch [30/53] - Loss: 14.0625
Epoch [2/2] - Batch [40/53] - Loss: 26.5625
Epoch [2/2] - Batch [50/53] - Loss: 25.0
Epoch [2/2] - Loss: 16.922169811320753


In [ ]:
trainer4 = Model4Trainer(learning_rate=0.01, num_epochs=2, batch_size=64)
trainer4.train(train_dataset)

Epoch [1/2] - Batch [0/53] - Loss: 0.7206496000289917
Epoch [1/2] - Batch [10/53] - Loss: 13.188368797302246
Epoch [1/2] - Batch [20/53] - Loss: 1.5058614015579224
Epoch [1/2] - Batch [30/53] - Loss: 0.13580989837646484
Epoch [1/2] - Batch [40/53] - Loss: 0.18023262917995453
Epoch [1/2] - Batch [50/53] - Loss: 0.11907884478569031
Epoch [1/2] - Loss: 9.057977744149712
Epoch [2/2] - Batch [0/53] - Loss: 0.3830423951148987
Epoch [2/2] - Batch [10/53] - Loss: 0.1684369444847107
Epoch [2/2] - Batch [20/53] - Loss: 0.08164428919553757
Epoch [2/2] - Batch [30/53] - Loss: 0.34158799052238464
Epoch [2/2] - Batch [40/53] - Loss: 0.21139264106750488
Epoch [2/2] - Batch [50/53] - Loss: 0.08206826448440552
Epoch [2/2] - Loss: 0.16007682807602971


## Test the Trained Network

In [ ]:
class ModelTester:
    def __init__(self, model, test_dataset, batch_size=64):
        self.model = model
        self.testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def test(self):
        self.model.eval()  # Set the model to evaluation mode
        correct = 0
        total = 0

        with torch.no_grad():  # Inference without gradient calculation
            for images, labels in self.testloader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)

                # Convert outputs/probabilities to predicted class (0 or 1)
                predicted = (outputs.data > 0.5).float()  # Threshold at 0.5
                total += labels.size(0)
                correct += (predicted == labels.unsqueeze(1)).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

In [ ]:
tester = ModelTester(trainer.model, test_dataset, batch_size=64)
tester.test()

Accuracy of the model on the test images: 98.57%


In [ ]:
tester2 = ModelTester(trainer2.model, test_dataset, batch_size=64)
tester2.test()

Accuracy of the model on the test images: 84.40%


In [ ]:
tester4 = ModelTester(trainer4.model, test_dataset, batch_size=64)
tester4.test()

Accuracy of the model on the test images: 98.33%
